In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [3]:
# Tables Used:

# - korepss_opportunities
# - cth_v_historical_ticket
# - cth_historical_attendance
# - ct_customer (& all extra tables)
# - flateamshop
# - location_ticket_type 
    # location_ticket_type_agg AS (
    # SELECT
    #     purchaser_ticketing_id,
    #     arena_level_internal
    # FROM
    #     (SELECT
    #          purchaser_ticketing_id,
    #          arena_level_internal,
    #          ROW_NUMBER() OVER (PARTITION BY purchaser_ticketing_id ORDER BY num_tickets DESC,
    #              CASE arena_level_internal
    #                  WHEN 'Premium' THEN 1
    #                  WHEN 'Lower' THEN 2
    #                  WHEN 'Club' THEN 3
    #                  ELSE 4
    #              END) AS rn
    #      FROM
    #          location_ticket_type)
    # WHERE rn = 1

In [4]:
q = """
SELECT DISTINCT
    purchaser_ticketing_id AS crm_id
FROM
    custom.cth_v_historical_ticket ticket
UNION
SELECT DISTINCT
    ticketing_account_scanned AS crm_id
FROM
    custom.cth_v_historical_attendance

"""

base_df = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)

In [5]:
q = """
SELECT
    purchaser_ticketing_id AS crm_id,
    SUM(gross_revenue) AS total_hockey_spend,
    SUM(paid_seats) AS total_hockey_seats,
    SUM(CASE WHEN season = '2025-26' THEN gross_revenue END) AS hockey_spend_2526,
    SUM(CASE WHEN season = '2025-26' THEN paid_seats END) AS hockey_seats_2526,
    datediff(day,MAX(transaction_datetime),current_date) AS days_since_last_purchase,
    COUNT(DISTINCT transaction_datetime) AS num_hockey_transactions
FROM
    custom.cth_v_historical_ticket
GROUP BY
    purchaser_ticketing_id
"""

ticket_df = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)

In [6]:
df = base_df.merge(ticket_df, how = 'left', on= 'crm_id')

In [7]:
q = """
SELECT
    ticketing_account_scanned AS crm_id,
    COUNT(*) AS total_game_attendance,
    COUNT(DISTINCT event_datetime) AS num_games_attended,
    COUNT(CASE WHEN season = '2025-26' THEN event_datetime END) AS hockey_tickets_attendance_2526,
    COUNT(DISTINCT CASE WHEN season = '2025-26' THEN event_datetime END) AS num_games_attendance_2526,
    datediff(day,MAX(event_datetime),current_date) AS days_since_last_attendance
FROM
    custom.cth_v_historical_attendance
GROUP BY
    ticketing_account_scanned
"""

attendance_df = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)

In [8]:
df = df.merge(attendance_df, how = 'left', on= 'crm_id')

In [9]:
q = """
SELECT
    purchaser_ticketing_id AS crm_id,
    LISTAGG(DISTINCT ticket_type, ', ') AS all_plan_types,
    LISTAGG(DISTINCT CASE WHEN season = '2025-26' THEN ticket_type END, ', ') AS plan_types_2526
FROM
    custom.cth_v_historical_plans
GROUP BY
    purchaser_ticketing_id
"""

plans_df = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)

In [10]:
df = df.merge(plans_df, how = 'left', on= 'crm_id')

In [11]:
q = """
SELECT DISTINCT
    clients.crm_id,
    email,
    CASE WHEN addresses.is_local = TRUE THEN 1 ELSE 0 END AS is_local
FROM
    custom.seatgeek_v_clients clients
INNER JOIN
    custom.golden_record_v_addresses addresses USING (email) 
"""

is_local_df = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)

In [12]:
df = df.merge(is_local_df, how = 'left', on= 'crm_id')

In [13]:
q = """
SELECT
    tradable_bits_activities.email,
    crm_id,
    COUNT(tradable_bits_activities.*) AS num_online_activities,
    COUNT(CASE WHEN creation_date >= '2025-07-01' THEN 1 END) AS num_online_activities_last_fiscal
FROM
    custom.tradable_bits_activities
LEFT JOIN
    custom.seatgeek_v_clients ON tradable_bits_activities.email = seatgeek_v_clients.email
GROUP BY
    tradable_bits_activities.email,
    crm_id
"""

tradable_bits_df = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)

In [14]:
df = df.merge(tradable_bits_df, how = 'left', on= 'crm_id')

In [15]:
q = """
SELECT
    crm_id,
    count(*) AS num_recieved,
    sum(CASE WHEN event = 'Opened' AND is_true_false THEN 1 ELSE 0 END) AS num_opened,
    sum(CASE WHEN event = 'Clicked' AND is_true_false THEN 1 ELSE 0 END) AS num_clicked,
    sum(CASE WHEN event = 'Bounced' AND is_true_false THEN 1 ELSE 0 END) AS num_bounced,
    sum(CASE WHEN event = 'Unsubscribed' AND is_true_false THEN 1 ELSE 0 END) AS num_unsubscribed,
    CASE WHEN max(date_unsubscribed_panthers) IS NOT NULL THEN True ELSE False END AS is_unsubscribed
FROM
    custom.sfmc_v_sent_reporting
LEFT JOIN
    custom.sfmc_v_subscribers ON sfmc_v_sent_reporting.subscriberkey = sfmc_v_subscribers.subscriber_key
LEFT JOIN
    custom.seatgeek_v_clients ON sfmc_v_subscribers.email = seatgeek_v_clients.email
WHERE
    sent_date >= '2025-07-01'
GROUP BY
    crm_id
"""

emails_df = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)

In [16]:
df = df.merge(emails_df, how = 'left', on= 'crm_id')

In [19]:
cols_999 = ['days_since_last_purchase', 'days_since_last_attendance']

df[cols_999] = df[cols_999].fillna(9999)

df = df.fillna(0)

df

,crm_id,total_hockey_spend,total_hockey_seats,hockey_spend_2526,hockey_seats_2526,days_since_last_purchase,num_hockey_transactions,total_game_attendance,num_games_attended,hockey_tickets_attendance_2526,...,is_local,email_y,num_online_activities,num_online_activities_last_fiscal,num_recieved,num_opened,num_clicked,num_bounced,num_unsubscribed,is_unsubscribed
0,22592444.0,1.381052e+05,792.0,30379.44,168.0,223.0,24.0,18.0,7.0,4.0,...,1.0,0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,False
1,21487314.0,1.224700e+07,121161.0,0.00,0.0,601.0,28920.0,207.0,70.0,0.0,...,1.0,0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,False
2,15582348.0,1.650000e+02,5.0,0.00,0.0,1199.0,1.0,9.0,2.0,0.0,...,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,21520918.0,4.728713e+04,464.0,6713.18,86.0,86.0,68.0,323.0,191.0,40.0,...,1.0,ericstreimer@yahoo.com,22.0,0.0,60.0,6.0,0.0,0.0,0.0,True
4,15633165.0,7.500000e+01,1.0,0.00,0.0,1186.0,1.0,3.0,2.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546666,26187982.0,0.000000e+00,0.0,0.00,0.0,9999.0,0.0,1.0,1.0,1.0,...,1.0,0,0.0,0.0,8.0,0.0,2.0,0.0,0.0,False
546667,26188737.0,0.000000e+00,0.0,0.00,0.0,9999.0,0.0,1.0,1.0,1.0,...,1.0,0,0.0,0.0,8.0,1.0,0.0,0.0,0.0,False
546668,26069455.0,0.000000e+00,0.0,0.00,0.0,9999.0,0.0,1.0,1.0,1.0,...,1.0,0,0.0,0.0,36.0,5.0,0.0,0.0,0.0,False
546669,26188861.0,0.000000e+00,0.0,0.00,0.0,9999.0,0.0,1.0,1.0,1.0,...,0.0,0,0.0,0.0,8.0,1.0,0.0,0.0,0.0,False
